#makemore: becoming a backprop ninja

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2025-06-24 15:07:15--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.004s  

2025-06-24 15:07:15 (56.9 MB/s) - ‘names.txt’ saved [228145/228145]



In [ ]:
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(w) for w in words)
print(words[:8])

32033
<generator object <genexpr> at 0x7c8fa1f958c0>
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [ ]:
block_size = 3

def build_dataset(words):
  X, Y = [], []


  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]



  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y


import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9 * len(words))


Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex) :5s} | approximite: {str(app):5s} | maxdiff: {maxdiff}')

In [ ]:
n_embed = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator=g)
#Layer1
W1 = torch.randn((n_embed * block_size, n_hidden), generator=g) * (5/3)/((n_embed * block_size) **0.5)
b1 = torch.randn(n_hidden, generator = g) * 0.1
#Layer2
W2 = torch.randn((n_hidden, vocab_size), generator = g) * 0.1
b2 = torch.randn(vocab_size, generator = g) * 0.1
#BatchNorm parametrs
bngain = torch.randn((1, n_hidden)) *0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parametrs = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parametrs))
for p in parametrs:
  p.requires_grad = True

4137


In [ ]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator = g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [ ]:
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
#Linear Layer 1
hprebn = embcat @ W1 + b1
#BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim = True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff ** 2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim = True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non - linearity
h = torch.tanh(hpreact)
logits = h @ W2 + b2
#cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim = True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims = True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()


for p in parametrs:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
          t.retain_grad()
loss.backward()
loss

tensor(3.3537, grad_fn=<NegBackward0>)

In [ ]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
dprobs = (1.0/probs) * dlogprobs
dcounts_sum_inv = (dprobs * counts).sum(1, keepdim = True)
dcounts = dprobs * counts_sum_inv
dcounts_sum = (-counts_sum ** -2) * dcounts_sum_inv
dcounts+= torch.ones_like(counts) * dcounts_sum
dnorm_logits = counts * dcounts
dlogit_maxes =(-dnorm_logits).sum(1, keepdim = True)
dlogits = dnorm_logits.clone()
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
dh = dlogits @ W2. T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact = (1 - h ** 2) * dh
dbngain = (bnraw * dhpreact).sum(0, keepdim = True)
dbnraw = dhpreact * bngain
dbnbias = dhpreact.sum(0, keepdim = True)
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim = True)
dbnvar = (-0.5 * (bnvar + 1e-5) ** -1.5) * dbnvar_inv
dbndiff2 = (1.0/(n-1)) * torch.ones_like(dbndiff) * dbnvar
dbndiff += 2 * bndiff * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum()
dhprebn += (1/n) * torch.ones_like(hprebn) * dbnmeani
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for t in range(Xb.shape[0]):
  for k in range(Xb.shape[1]):
    ix = Xb[t, k]
    dC[ix] += demb[t, k]


def cmp(s, dt, t):
  ex = (t.grad is not None) and torch.all(dt == t.grad).item()
  app = (t.grad is not None) and torch.allclose(dt, t.grad)
  maxdiff = (t.grad is not None) and (dt - t.grad).abs().max().item() or float('inf')
  print(f'{s:15s} | exact: {str(ex) :5s} | approximite: {str(app):5s} | maxdiff: {maxdiff}')


cmp('logprobs', dlogprobs, logprobs)
cmp('dprobs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximite: True  | maxdiff: inf
dprobs          | exact: True  | approximite: True  | maxdiff: inf
counts_sum_inv  | exact: True  | approximite: True  | maxdiff: inf
counts          | exact: True  | approximite: True  | maxdiff: inf
norm_logits     | exact: True  | approximite: True  | maxdiff: inf
logit_maxes     | exact: True  | approximite: True  | maxdiff: inf
logits          | exact: True  | approximite: True  | maxdiff: inf
h               | exact: True  | approximite: True  | maxdiff: inf
W2              | exact: True  | approximite: True  | maxdiff: inf
b2              | exact: True  | approximite: True  | maxdiff: inf
hpreact         | exact: False | approximite: True  | maxdiff: 9.313225746154785e-10
bngain          | exact: False | approximite: True  | maxdiff: 1.862645149230957e-09
bnbias          | exact: False | approximite: True  | maxdiff: 3.725290298461914e-09
bnraw           | exact: False | approximite: True  | maxdiff: 9.3132257461

In [ ]:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.3537099361419678 diff: -2.384185791015625e-07


In [ ]:
dlogits = F.softmax(logits,1)
dlogits[range(n), Yb] -= 1
dlogits /= n
cmp('logits', dlogits, logits)

logits          | exact: False | approximite: True  | maxdiff: 6.51925802230835e-09


In [ ]:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)
